## Frequent Pattern Mining

Bước 1: Tiền xử lý dữ liệu ảnh. Lấy tên tệp và gán nó làm label cho tất cả các ảnh trong tệp đó. Chỉ sử dụng những ảnh đuôi .pgm.

In [8]:
import os
import cv2

ROOT_DIR = os.path.abspath("../")
data_dir = os.path.join(ROOT_DIR, "CMU_FACE_Data/faces")
X = []  # Features
y = []  # Labels
size = (64, 64)

for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)
    if not os.path.isdir(subdir_path):
        continue

    for file_name in os.listdir(subdir_path):
        if not file_name.endswith(".pgm"):
            continue

        parts = file_name.split("_")
        user_id = parts[0]

        file_path = os.path.join(subdir_path, file_name)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        if image is not None:
            X.append(image)
            y.append(subdir)

In [9]:
import numpy as np

print(np.unique(y))

['an2i' 'at33' 'boland' 'bpm' 'ch4f' 'cheyer' 'choon' 'danieln' 'glickman'
 'karyadi' 'kawamura' 'kk49' 'megak' 'mitchell' 'night' 'phoebe' 'saavik'
 'steffi' 'sz24' 'tammo']


Bước 2: Triển khai một lớp gọi là "LocalBinaryPatterns" và một phương thức "describe" để tính toán biểu diễn Local Binary Pattern (LBP) của một hình ảnh.
Phương thức "describe" nhận vào một hình ảnh và thực hiện các bước để tính toán biểu diễn LBP của hình ảnh đó. Các bước chính gồm:

1. Sử dụng hàm "local_binary_pattern" từ module "feature" để tính toán LBP của hình ảnh. Các đối số của hàm bao gồm:
"image": hình ảnh đầu vào.
"numPoints": số lượng điểm mẫu sử dụng trong quá trình tính toán LBP.
"radius": bán kính của vùng lân cận xung quanh mỗi điểm mẫu.
"method": phương pháp tính toán LBP, trong trường hợp này là "uniform".
Kết quả của bước này là một ma trận LBP.

2. Sử dụng hàm "histogram" từ thư viện NumPy để xây dựng histogram của các mẫu LBP. Các đối số của hàm bao gồm:
"lbp.ravel()": một mảng 1D chứa các giá trị LBP từ ma trận LBP.
"bins": các khoảng giá trị để chia histogram, trong trường hợp này là từ 0 đến numPoints + 3.
"range": khoảng giá trị của histogram, trong trường hợp này là từ 0 đến numPoints + 2.
Kết quả của bước này là histogram của các mẫu LBP.

Chuẩn hóa histogram bằng cách chia tất cả các giá trị trong histogram cho tổng của chúng cộng với một giá trị rất nhỏ (eps) để tránh chia cho 0.

Trả về histogram của Local Binary Pattern.

In [10]:
from skimage import feature

class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		self.numPoints = numPoints
		self.radius = radius

	def describe(self, image, eps=1e-7):
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))

		hist = hist.astype("float")
		hist /= (hist.sum() + eps)

		return hist

Bước 3: Tạo một đối tượng desc từ lớp LocalBinaryPatterns với numPoints = 64 và radius = 8. Đối tượng này sẽ được sử dụng để tính toán biểu diễn LBP của các hình ảnh.

Tiếp theo, ta khởi tạo một danh sách rỗng features để lưu trữ các đặc trưng LBP của các hình ảnh.

Sau đó, trong vòng lặp for x in X, với X là một danh sách chứa các hình ảnh đầu vào, ta thực hiện các bước sau:

1. Gọi phương thức describe của đối tượng desc để tính toán biểu diễn LBP của hình ảnh x. Kết quả được lưu vào biến f.
2. Kiểm tra xem f có khác None hay không. Nếu khác None, tức là tính toán LBP thành công, ta thêm f vào danh sách features.

Cuối cùng, sau khi vòng lặp kết thúc, danh sách features sẽ chứa các biểu diễn LBP của các hình ảnh trong X.

In [11]:
desc = LocalBinaryPatterns(64, 8)

features = []
for x in X:
    f = desc.describe(x)
    if f is not None:
        features.append(f)

Bước 4: Sử dụng phương thức train_test_split để chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỉ lệ 8 : 2.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

Bước 5: Sử dụng thư viện mlxtend để thực hiện phân tích mẫu phổ biến.

1. Tạo một mảng X_arr từ danh sách train_frequent_patterns và chuyển đổi thành mảng numpy bằng np.array().
2. Tạo DataFrame df từ mảng X_arr.
3. Sử dụng df.applymap(str) để chuyển đổi tất cả các phần tử của DataFrame thành kiểu dữ liệu chuỗi (string).
4. Sử dụng pd.get_dummies(df) để thực hiện mã hóa one-hot encoding trên DataFrame df. Kết quả là DataFrame one_hot_df với các cột được tạo ra từ các giá trị duy nhất trong DataFrame ban đầu.
5. Sử dụng fpgrowth(one_hot_df, min_support=0.05, use_colnames=True) để áp dụng thuật toán FP-Growth trên DataFrame one_hot_df. Các đối số:
    + min_support: ngưỡng hỗ trợ tối thiểu của mẫu phổ biến. Ở đây, ngưỡng là 0.05, tức là mẫu phổ biến phải xuất hiện ít nhất 5% trong tập dữ liệu.
    + use_colnames: sử dụng tên cột thay vì chỉ số của cột trong kết quả mẫu phổ biến.
    
Kết quả trả về là DataFrame frequent_patterns chứa tập hợp các mẫu phổ biến cùng với giá trị hỗ trợ của chúng. Mỗi hàng đại diện cho một tập hợp  phổ biến, và cột "itemsets" chứa các mẫu tạo thành tập hợp. Giá trị hỗ trợ đại diện cho tỷ lệ giao dịch trong tập dữ liệu chứa tập hợp mẫu.

In [13]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth

X_arr = np.array(X_train)
df = pd.DataFrame(X_arr)
df = df.applymap(str)

one_hot_df = pd.get_dummies(df)
frequent_patterns = fpgrowth(one_hot_df, min_support=0.05, use_colnames=True)

In [14]:
print(frequent_patterns)

       support                            itemsets
0     0.329993                            (54_0.0)
1     0.219773                            (48_0.0)
2     0.075484           (50_0.000260416666659885)
3     0.066800            (51_0.00052083333331977)
4     0.060120            (46_0.00052083333331977)
...        ...                                 ...
2422  0.052772  (26_0.0010416666665581597, 56_0.0)
2423  0.056780  (26_0.0010416666665581597, 54_0.0)
2424  0.050768  (26_0.0010416666665581597, 55_0.0)
2425  0.055444  (23_0.0020833333331163194, 54_0.0)
2426  0.057448  (34_0.0020833333331163194, 54_0.0)

[2427 rows x 2 columns]


Bước 6: Sử dụng mô hình RandomForestClassifier để phân loại.

1. Tạo một đối tượng clf_rf từ lớp RandomForestClassifier với các đối số:
    + n_estimators: số lượng cây quyết định trong mô hình Random Forest. Ở đây, ta đặt giá trị là 100.
    + random_state: giá trị để đảm bảo sự phân chia ngẫu nhiên nhưng nhất quán. Ở đây, ta đặt giá trị là 42.
2. Sử dụng phương thức fit(X_train, y_train) trên đối tượng clf_rf để huấn luyện mô hình RandomForestClassifier trên tập huấn luyện. Đối số X_train là tập đặc trưng huấn luyện và y_train là nhãn tương ứng.
3. Sử dụng phương thức predict(X_test) trên đối tượng clf_rf để dự đoán nhãn cho tập kiểm tra X_test. Kết quả được lưu vào biến y_pred.

Cuối cùng, ta có một mô hình RandomForestClassifier đã được huấn luyện và sử dụng để dự đoán nhãn cho tập kiểm tra X_test.

In [15]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)

Bước 7: Tính Accuracy và F1 Score.

+ Độ chính xác (Accuracy): Độ chính xác được tính bằng cách so sánh nhãn thực tế (y_test) với nhãn dự đoán (y_pred) trên tập kiểm tra. Độ chính xác là tỷ lệ giữa số lượng dự đoán chính xác và tổng số mẫu. acc = TP + TN / (TP + TN + FP + FN)
+ F1 Score: F1 Score là một số đo kết hợp giữa độ chính xác (precision) và độ phủ (recall) của mô hình. F1 Score là trung bình điều hoà của precision và recall, và được tính bằng công thức: F1 = 2 * (precision * recall) / (precision + recall).
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)

In [16]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.968


In [17]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='macro')
print("F1 Score:", f1)

F1 Score: 0.9624784623849021
